<a href="https://colab.research.google.com/github/tm-ty/colab-notebooks-tm/blob/main/%F0%9F%94%A5Crawler_em_Massa_(Crawl4AI)_(Parallel_Crawling_Batch)(working)_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# v5 - by Claude
**Instruções de Uso:**

1. Abra um novo notebook no Google Colab.
2. Copie e cole o código fornecido em uma célula do notebook.
3. Execute a célula (Ctrl+Enter ou clique no botão de play).
4. O código irá:
   - Instalar as dependências necessárias
   - Buscar automaticamente as URLs do sitemap da Pydantic AI
   - Rastrear cada página
   - Gerar arquivos Markdown
   - Criar um arquivo ZIP com todos os Markdowns
5. Ao final da execução, um arquivo `crawl4ai_output.zip` será disponibilizado para download.
6. Se solicitado, reinicie a sessão do Colab após a instalação das dependências.

**Personalização:**

Para usar em outro site, altere a URL do sitemap na linha:
```python
sitemap_url = "https://ai.pydantic.dev/sitemap.xml"
```

## v5 - by Claude - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

In [ ]:
# Instalar dependências
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Analisar o XML
        root = ElementTree.fromstring(response.content)

        # Extrair todas as URLs do sitemap
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Erro ao buscar o sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Rastreamento Sequencial com Reutilização de Sessão ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Rastreado com sucesso: {url}")
                print(f"Tamanho do Markdown: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown salvo em: {filename}")

            else:
                print(f"Falha: {url} - Erro: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml" # <<-- INSIRA SUA URL NESSA LINHA
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Encontradas {len(urls)} URLs para rastrear")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Diretório de saída compactado para crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")
    else:
        print("Nenhuma URL encontrada para rastrear")

# Executar a função principal
await main()

## v5 - by Claude - CÓDIGO COMPLETO **COM SEPARAÇÃO** DE CÉLULAS

### 1. Célula de dependências:

In [ ]:
# Instalar dependências
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

### 2. Células de código principal:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# ... (o restante do seu código)

### 3. Fluxo de trabalho recomendado:

In [ ]:
# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Analisar o XML
        root = ElementTree.fromstring(response.content)

        # Extrair todas as URLs do sitemap
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Erro ao buscar o sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Rastreamento Sequencial com Reutilização de Sessão ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Rastreado com sucesso: {url}")
                print(f"Tamanho do Markdown: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown salvo em: {filename}")

            else:
                print(f"Falha: {url} - Erro: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml" # <<-- INSIRA SUA URL NESSA LINHA
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Encontradas {len(urls)} URLs para rastrear")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Diretório de saída compactado para crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")
    else:
        print("Nenhuma URL encontrada para rastrear")

# Executar a função principal
await main()

# v5 by Gemini Exp 1206 - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

**Como executar:**

1. Execute a célula de instalação de dependências **uma vez**. (Se solicitado, reinicie a sessão do Colab após a instalação das dependências.)
2. Execute as demais células.
3. O código irá:
    *   Obter as URLs do `sitemap.xml`.
    *   Rastrear cada URL.
    *   Salvar os arquivos Markdown temporariamente em `/content/crawl4ai_temp_output`.
    *   Criar um arquivo `crawl4ai_output.zip` contendo todos os arquivos Markdown.
    *   Iniciar o download do arquivo `crawl4ai_output.zip` no seu navegador.

Agora você tem um processo automatizado que obtém as URLs do sitemap, rastreia cada uma delas e fornece os resultados em um arquivo zip para download!


## v5 by Gemini Exp 1206 - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

In [ ]:
# Instalar dependências (execute esta célula apenas uma vez)
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    """
    Fetches all URLs from a given sitemap URL.

    Args:
        sitemap_url: URL of the sitemap.

    Returns:
        List[str]: List of URLs found in the sitemap.
    """
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        # The namespace is usually defined in the root element
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching or parsing sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Sequential Crawling with Session Reuse ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
        # Sem output_dir, vamos salvar os arquivos temporariamente e depois zipá-los
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Successfully crawled: {url}")
                print(f"Markdown length: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown saved to: {filename}")

            else:
                print(f"Failed: {url} - Error: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml"  # URL do sitemap
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Found {len(urls)} URLs to crawl")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Output directory zipped to crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")

        # Limpar o diretório temporário (opcional)
        shutil.rmtree(temp_output_directory)
        print(f"Temporary output directory removed: {temp_output_directory}")
    else:
        print("No URLs found to crawl")

# Executar a função principal
await main()

## v5 by Gemini Exp 1206 - CÓDIGO COMPLETO **COM SEPARAÇÃO** DE CÉLULAS

### 1. Célula de dependências:

In [ ]:
# Instalar dependências (execute esta célula apenas uma vez)
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

### 2. Células de código principal:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# ... (o restante do seu código)

### 3. Fluxo de trabalho recomendado:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    """
    Fetches all URLs from a given sitemap URL.

    Args:
        sitemap_url: URL of the sitemap.

    Returns:
        List[str]: List of URLs found in the sitemap.
    """
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        # The namespace is usually defined in the root element
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching or parsing sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Sequential Crawling with Session Reuse ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
        # Sem output_dir, vamos salvar os arquivos temporariamente e depois zipá-los
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Successfully crawled: {url}")
                print(f"Markdown length: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown saved to: {filename}")

            else:
                print(f"Failed: {url} - Error: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml"  # URL do sitemap
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Found {len(urls)} URLs to crawl")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Output directory zipped to crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")

        # Limpar o diretório temporário (opcional)
        shutil.rmtree(temp_output_directory)
        print(f"Temporary output directory removed: {temp_output_directory}")
    else:
        print("No URLs found to crawl")

# Executar a função principal
await main()